In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

from matplotlib.pylab import rcParams
rcParams['figure.figsize']=20,10

from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Input, Activation, concatenate
from keras import optimizers
from keras.models import Model
from pandas_datareader import data as wb
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

Using TensorFlow backend.
C:\Users\vucht\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\vucht\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\vucht\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\vucht\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [2]:
n = 120
n_day = 5
ticker =  'OUT1V.HE'
start = '2013-1-1'
end = '2020-07-15'
t = datetime.strptime(end, '%Y-%m-%d').date()+ timedelta(days = n)
end_predict = t.strftime("%Y-%m-%d")
k=60
data = pd.DataFrame()
data = wb.DataReader(ticker, data_source = 'yahoo', start = start, end = end)
feature = data.iloc[:,0:5]
feature1 = np.array([np.mean(data["Adj Close"].values[i:i+n_day]) for i in range(len(data)-n_day)])
feature2 = np.array([np.mean(data["Close"].values[i:i+k]) for i in range(len(data)-k)])
price = data["Adj Close"].values

In [3]:
x_norm = MinMaxScaler()
x_norm.fit(feature)
X = x_norm.transform(feature)

y_norm = MinMaxScaler()
y_norm.fit(price.reshape(-1,1))
y = y_norm.transform(price.reshape(-1,1))

x1_norm = MinMaxScaler()
x1_norm.fit(feature1.reshape(-1,1))
X1 = x1_norm.transform(feature1.reshape(-1,1))

x2_norm = MinMaxScaler()
x2_norm.fit(feature2.reshape(-1,1))
X2 = x2_norm.transform(feature2.reshape(-1,1))

In [4]:
X = np.array([X[i:k+i] for i in range(len(X)-k)])
X1 = np.array([X1[k+i-n_day] for i in range(len(X1)-k+n_day)])
X2 = np.array(X2)
y = np.array([y[k+i] for i in range(len(y)-k)])

In [5]:
n = int(len(X)*0.8)
x_train = X[:n]
x_test = X[n:]
y_train = y[:n]
y_test = y[n:]
x1_train = X1[:n]
x1_test = X1[n:]
x2_train = X2[:n]
x2_test = X2[n:]

In [6]:
# define two sets of inputs
lstm_input = Input(shape=(k, 5), name='lstm_input')
dense_input = Input(shape=(X1.shape[1],), name='tech_input')
 
# the first branch operates on the first input
a = LSTM(50, name='lstm_0')(lstm_input)
a = Dropout(0.2, name='lstm_dropout_0')(a)
lstm_branch = Model(inputs=lstm_input, outputs=a)
 
# the second branch opreates on the second input
b = Dense(20, name='tech_dense_0')(dense_input)
b = Activation("relu", name='tech_relu_0')(b)
b = Dropout(0.2, name='tech_dropout_0')(b)
technical_indicators_branch = Model(inputs=dense_input, outputs=b)
 
# combine the output of the two branches
combined = concatenate([lstm_branch.output, technical_indicators_branch.output], name='concatenate')
 
c = Dense(64, activation="sigmoid", name='dense_pooling')(combined)
c = Dense(1, activation="linear", name='dense_out')(c)
 
# our model will accept the inputs of the two branches and then output a single value
model1 = Model(inputs=[lstm_branch.input, technical_indicators_branch.input], outputs=c)

adam = optimizers.Adam(lr=0.0005)

model1.compile(optimizer=adam,
              loss='mse')

In [7]:
model1.fit(x=[x_train, x2_train], y=y_train, batch_size=32, epochs=50, verbose = 0, shuffle=True, validation_split=0.1)

y_predicted = model1.predict([x_test,x2_test])
f1 = y_norm.inverse_transform(y_predicted)

y_real = y_norm.inverse_transform(y_test)
plt.gcf().set_size_inches(22, 15, forward=True)

start = 0
end = -1

real = plt.plot(y_real[start:end], label='real')
pred1 = plt.plot(f1[start:end], label='F1')
plt.legend(['Real', 'F1'])

plt.show()

In [8]:
x_date = X[-4:]
x2_date = X2[-4:]

In [9]:
y_date_predicted = model1.predict([x_date,x2_date])
y_date_predicted
print(y_norm.inverse_transform(y_date_predicted))

[[2.2495937]
 [2.232863 ]
 [2.236729 ]
 [2.2358243]]


In [10]:
model2 = Model(inputs=[lstm_branch.input, technical_indicators_branch.input], outputs=c)

adam = optimizers.Adam(lr=0.0005)

model2.compile(optimizer=adam,
              loss='mse')


In [11]:
model2.fit(x=[x_train, x2_train], y=y_train, batch_size=8, verbose = 0, epochs=50, shuffle=True, validation_split=0.1)

y_predicted = model2.predict([x_test,x2_test])
f2 = y_norm.inverse_transform(y_predicted)

y_real = y_norm.inverse_transform(y_test)
plt.gcf().set_size_inches(22, 15, forward=True)

start = 0
end = -1

real = plt.plot(y_real[start:end], label='real')
#pred1= plt.plot(f1[start:end], label='F1')
pred2 = plt.plot(f2[start:end], label='F2')
plt.legend(['Real', 'F2'])

plt.show()

In [12]:
x_date = X[-4:]
x2_date = X2[-4:]

In [13]:
y_date_predicted = model2.predict([x_date,x2_date])
y_date_predicted
print(y_norm.inverse_transform(y_date_predicted))

[[1.903331 ]
 [1.8535339]
 [1.918528 ]
 [1.8948383]]


In [14]:
model3 = Model(inputs=[lstm_branch.input, technical_indicators_branch.input], outputs=c)

adam = optimizers.Adam(lr=0.0005)

model3.compile(optimizer=adam,
              loss='mse')


In [15]:
model3.fit(x=[x_train, x2_train], y=y_train, batch_size=2, verbose = 2, epochs=50, shuffle=True, validation_split=0.1)

Train on 1316 samples, validate on 147 samples
Epoch 1/50
 - 17s - loss: 0.0013 - val_loss: 0.0029
Epoch 2/50
 - 18s - loss: 0.0013 - val_loss: 3.4105e-04
Epoch 3/50
 - 17s - loss: 0.0012 - val_loss: 3.3080e-04
Epoch 4/50
 - 18s - loss: 0.0014 - val_loss: 4.3745e-04
Epoch 5/50
 - 18s - loss: 0.0012 - val_loss: 0.0019
Epoch 6/50
 - 15s - loss: 0.0012 - val_loss: 0.0019
Epoch 7/50
 - 15s - loss: 0.0012 - val_loss: 3.3722e-04
Epoch 8/50
 - 15s - loss: 0.0012 - val_loss: 0.0016
Epoch 9/50
 - 15s - loss: 0.0013 - val_loss: 0.0017
Epoch 10/50
 - 15s - loss: 0.0012 - val_loss: 0.0024
Epoch 11/50
 - 15s - loss: 0.0013 - val_loss: 0.0022
Epoch 12/50
 - 15s - loss: 0.0012 - val_loss: 9.9600e-04
Epoch 13/50
 - 15s - loss: 0.0011 - val_loss: 4.7172e-04
Epoch 14/50
 - 15s - loss: 0.0013 - val_loss: 9.4273e-04
Epoch 15/50
 - 15s - loss: 0.0012 - val_loss: 3.2477e-04
Epoch 16/50
 - 15s - loss: 0.0013 - val_loss: 0.0011
Epoch 17/50
 - 15s - loss: 0.0013 - val_loss: 8.9960e-04
Epoch 18/50
 - 15s - loss

y_predicted = model3.predict([x_test,x2_test])
f3 = y_norm.inverse_transform(y_predicted)

y_real = y_norm.inverse_transform(y_test)
plt.gcf().set_size_inches(22, 15, forward=True)

start = 0
end = -1

real = plt.plot(y_real[start:end], label='real')
#pred1= plt.plot(f1[start:end], label='F1')
pred2 = plt.plot(f3[start:end], label='F3')
plt.legend(['Real', 'F3'])

plt.show()

In [16]:
x_date = X[-4:]
x2_date = X2[-4:]

In [17]:
y_date_predicted = model2.predict([x_date,x2_date])
y_date_predicted
print(y_norm.inverse_transform(y_date_predicted))

[[2.2553132]
 [2.1979995]
 [2.2955647]
 [2.232114 ]]
